In [17]:
# Importing the necessary libraries

# GraphDatabase is used to interact with Neo4j databases.
from neo4j import GraphDatabase  

# Folium is used for creating interactive maps in Python.
import folium  

# Geocoder helps in geocoding (getting latitude and longitude from addresses).
import geocoder  

# Requests is used to make HTTP requests (used for external APIs).
import requests  

# Geopy's geodesic function is used for calculating the geographical distance between two points.
from geopy.distance import geodesic  

# Nominatim is a geocoding service in Geopy that converts an address to coordinates and vice versa.
from geopy.geocoders import Nominatim  

# Neo4j GraphDatabase module is imported again, possibly redundant as it's already imported above.
from neo4j import GraphDatabase  

# tkinter is used for building graphical user interfaces (GUIs) in Python.
import tkinter as tk  


In [18]:
def split_coordinates(coord_string):
    # Split the input string by the '|' character to separate coordinate pairs
    pairs = coord_string.split('|')
    
    # Initialize a set to store unique coordinates (sets handle duplicates)
    coordinates = set()
    
    # Loop through each pair in the list of pairs
    for pair in pairs:
        # Split each pair by '#' to separate latitude and longitude
        lat_long = pair.split('#')
        
        # If the pair contains exactly two elements (latitude and longitude)
        if len(lat_long) == 2:
            try:
                # Convert both latitude and longitude to floats
                longitude = float(lat_long[0])
                latitude = float(lat_long[1])
                
                # Add the tuple (latitude, longitude) to the set to ensure uniqueness
                coordinates.add((latitude, longitude))
            
            except ValueError:
                # Handle cases where the conversion to float fails
                print(f"Error converting values for pair: {pair}")
    
    # Convert the set back to a list to maintain consistent output format
    return list(coordinates)


In [19]:
from neo4j import GraphDatabase

# Informations de connexion
uri = "bolt://localhost:7687"  # URI pour la connexion à Neo4j
user = "oussa"  # Nom d'utilisateur pour l'authentification
password = "oussa1234"  # Mot de passe pour l'authentification

# Connexion à Neo4j avec gestion de ressources
# Utilisation du 'with' pour assurer la fermeture automatique des ressources
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    # Ouverture d'une session
    # L'utilisation du 'with' garantit que la session est fermée automatiquement
    with driver.session(database="pharma") as session:
        # Requête Cypher pour projeter un graphe pour les algorithmes de cheminement
        query = '''
        CALL gds.graph.project(
            'myGraph',  
            'Location',  
            'ROAD',  
            {
                relationshipProperties: 'length'  
            }
        )
        '''
        
        # Exécution de la requête pour projeter le graphe
        session.run(query)

# Ressources fermées automatiquement grâce au 'with' statement


In [20]:
from neo4j import GraphDatabase

def find_nearest_pharmacy(uri, user, password, latitude, longitude):
    # Connexion au driver Neo4j avec gestion des ressources
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    try:
        # Ouverture de la session Neo4j avec gestion automatique de la fermeture
        with driver.session(database="pharma") as session:
            # Requête pour trouver le noeud le plus proche
            query = '''
            MATCH (n)
            WITH n, point.distance(point({latitude: $latitude, longitude: $longitude}), point({latitude: n.y, longitude: n.x})) AS dist
            RETURN n, n.osmid AS nearest_osmid, n.x AS nearest_latitude, n.y AS nearest_longitude
            ORDER BY dist
            LIMIT 1;
            '''
            # Exécution de la requête avec les paramètres passés (latitude et longitude)
            nearest_result = session.run(query, latitude=latitude, longitude=longitude).single()
            
            if nearest_result is None:
                # Si aucun noeud n'est trouvé, renvoyer un message approprié
                return "No nodes found"
            
            nearest_osmid = nearest_result["nearest_osmid"]
            
            # Requête pour trouver le chemin le plus court vers une pharmacie
            combined_query = '''
            MATCH (source:Location {osmid: $nearest_osmid})
            CALL gds.allShortestPaths.dijkstra.stream('myGraph', {
                sourceNode: source,
                relationshipWeightProperty: 'length'
            })
            YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
            WHERE gds.util.asNode(targetNode).pharmacy IS NOT NULL
            WITH
                index,
                gds.util.asNode(sourceNode).osmid AS sourceNodeName,
                gds.util.asNode(targetNode).osmid AS targetNodeName,
                totalCost,
                [nodeId IN nodeIds | gds.util.asNode(nodeId).osmid] AS nodeNames,
                costs,
                nodes(path) AS path,
                relationships(path) AS rels
            ORDER BY totalCost
            LIMIT 1
            WITH nodeNames, [rel IN rels | rel.geometry] AS geometries
            UNWIND range(0, size(nodeNames) - 2) AS idx
            MATCH (n1 {osmid: nodeNames[idx]})-[r]->(n2 {osmid: nodeNames[idx + 1]})
            RETURN collect(properties(r).geometry) AS relationshipProperties
            '''
            
            # Exécution de la requête combinée
            result = session.run(combined_query, nearest_osmid=nearest_osmid).single()
            
            if result is None:
                # Si aucun chemin vers une pharmacie n'est trouvé
                return "No path to pharmacy found"
            
            # Récupération et concaténation des propriétés des relations
            relationship_properties = result["relationshipProperties"]
            chemin = ''.join(relationship_properties)
            
            # Retourner le chemin trouvé
            return chemin
    
    except Exception as e:
        # Gestion des erreurs et renvoi du message d'erreur
        return str(e)
    
    finally:
        # Fermeture du driver Neo4j dans le bloc finally pour garantir sa fermeture
        driver.close()


In [21]:
import geocoder

def get_current_location():
    try:
        # Utiliser geocoder pour obtenir l'emplacement basé sur l'adresse IP
        location = geocoder.ip('')  # Adresse IP vide pour récupérer l'emplacement public
        if location.latlng:  # Vérifie si la latitude et la longitude sont disponibles
            return location.latlng  # Retourne une liste avec la latitude et la longitude
        else:
            # Si l'emplacement n'est pas trouvé, renvoyer un message approprié
            print("Unable to retrieve location")
            return None
    except Exception as e:
        # Gérer les erreurs qui peuvent survenir avec geocoder
        print(f"Error occurred while retrieving location: {e}")
        return None

# Appel de la fonction pour obtenir l'emplacement actuel
current_location = get_current_location()

# Vérifier si un emplacement a été récupéré
if current_location:
    latitude, longitude = current_location
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("Could not retrieve current location.")


Latitude: 34.0331, Longitude: -5.0003


In [22]:
uri = "bolt://localhost:7687"
user = "oussa"
password = "oussa1234"
latitude = 34.016485
longitude = -5.033121

In [23]:
chemin = find_nearest_pharmacy(uri, user, password, latitude, longitude)

In [24]:
chemin

'|-5.0330673#34.0163942|-5.0332311#34.0163393|-5.0332311#34.0163393|-5.0334136#34.016677|-5.0334136#34.016677|-5.033541181730647#34.016630886121455|-5.0335195#34.0165709'

In [25]:
a = split_coordinates(chemin)

In [26]:
a = [(latitude,longitude)] + a

In [27]:
import folium

# Créer une carte centrée sur un emplacement spécifique
maps = folium.Map(location=a[0], zoom_start=50)

# Ajouter l'itinéraire à la carte
folium.Marker(location=a[0], icon=folium.Icon(color="green",icon='home')).add_to(maps)
folium.Marker(location=a[-1], icon=folium.Icon(color="red",icon='flag')).add_to(maps)
folium.PolyLine(a, color='blue', weight=5, opacity=0.5).add_to(maps)

# Afficher la carte
maps